<a href="https://colab.research.google.com/github/nxxk23/NAKBID/blob/main/NINK/SearchCheckdrug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
!pip -q install gradio neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
from neo4j import GraphDatabase
import os
import gradio as gr
import requests
import json
import csv
from io import StringIO

In [ ]:
API_SEARCH_URL = "http://quanta3.manageai.co.th:7779/search"
API_INTERACTION_URL = "http://quanta2.manageai.co.th:7777/interactions/"

# Function to call the FastAPI endpoint and get drug interactions
def get_interactions(drug_names, tpuid_ids):
    data = {
        "input_text": drug_names.split(","),
        "tpu_id": [int(x) for x in tpuid_ids.split(",")]
    }
    response = requests.post(API_INTERACTION_URL, json=data)

    if response.status_code == 200:
        interactions_data = response.json().get("interactions", [])
        interaction_pairs = []
        # Prepare a list of drug pairs to show in the dropdown
        for interaction in interactions_data:
            pair_name = f"{interaction['drug_name1']} ⛔ {interaction['drug_name2']}"
            interaction_pairs.append(pair_name)  # Just names for dropdown
        return interaction_pairs, interactions_data  # Return pairs and full interaction data
    else:
        print(f"Error fetching data: {response.status_code}")
        return [], []

In [ ]:
def on_submit(drug_names, tpuid_ids):
    interactions, interaction_data = get_interactions(drug_names, tpuid_ids)
    print(f"Fetched interactions: {interactions}")
    print(f"Interaction Data: {interaction_data}")
    if not interactions:
        return (
            gr.update(choices=["No opposing drug interactions"], value="No opposing drug interactions"),  # Show message in dropdown
            "No opposing drug interactions",  # Show error message
            gr.State([]),
            "", "", "", "", "", "", "", ""
        )
    return (
        gr.update(choices=interactions, value=""),
        "",
        gr.State(interaction_data),
        "", "", "", "", "", "", "", ""
    )

def store_detail(interaction_data):
    # If there's no interaction data, return empty fields
    if not interaction_data:
        print("No interaction data available.")  # Log when no data exists
        return [["", "", "", "", "", "", ""]]  # Return an empty set of values
    all_details = []
    # Iterate over all interaction data and collect the details
    for details in interaction_data:
        print(f"Found details: {details}")  # Log the found details
        # Collect the details for each interaction, defaulting to empty string if not available
        interaction_details = [
            details.get("Significance", ""),
            details.get("Onset", ""),
            details.get("Severity", ""),
            details.get("Documentation", ""),
            details.get("interaction_detail",""),
            details.get("mechanism", ""),
            details.get("management", ""),
            details.get("discussion", ""),
        ]
        all_details.append(interaction_details)
    return all_details

def on_interaction_select(selected_interaction, drug_input, tpu_input):
    print(f"Selected interaction: {selected_interaction}")  # Log the selected interaction
    interactions, interaction_detail = get_interactions(drug_input, tpu_input)
    for idx, interaction in enumerate(interactions):
        if selected_interaction == interaction:
            print(f"Found matching interaction: {selected_interaction}")  # Log when a match is found
            # Extract the corresponding interaction_detail for the matched interaction
            matching_interaction_detail = interaction_detail[idx]
            print(f"Interaction detail: {matching_interaction_detail}")  # Log the corresponding interaction details
            # Store the matching details using store_detail function
            my_store = store_detail([matching_interaction_detail])  # จะได้ list ซ้อน [[...]]
            print(tuple(my_store[0]))
            # ต้องแยกค่าออกมาเป็น 8 ตัว ไม่ใช่ list ซ้อน
            return tuple(my_store[0])  # ใช้ tuple เพื่อให้แยกค่าเป็น 8 ตัว

In [ ]:
def search_suggestions(query):
    uri = "neo4j+s://test-neo4j.manageai.co.th:7687"
    user = "neo4j"
    password = "iRU8ElhtDhoe0XdM72c0muzWY"

    driver = GraphDatabase.driver(uri, auth=(user, password))
    suggestions = set()

    with driver.session() as session:
        result = session.run("""
            MATCH (d:DRUG)
            WHERE
                toLower(d.`TMTID(TPU)`) CONTAINS toLower($q) OR
                toLower(d.`TMTID(TP)`) CONTAINS toLower($q) OR
                toLower(d.`TMTID(GPU)`) CONTAINS toLower($q) OR
                toLower(d.`TMTID(GP)`) CONTAINS toLower($q) OR
                toLower(d.`GPUNAME`) CONTAINS toLower($q) OR
                toLower(d.`GPNAME`) CONTAINS toLower($q) OR
                toLower(d.`TPUNAME`) CONTAINS toLower($q) OR
                toLower(d.`TPNAME`) CONTAINS toLower($q) OR
                toLower(d.`VTMNAME`) CONTAINS toLower($q) OR
                any(sub IN d.`SUBS_LIST` WHERE toLower(sub) CONTAINS toLower($q))
            RETURN DISTINCT d.`TPUNAME` AS label
            LIMIT 20
        """, q=query)

        for record in result:
            if record["label"]:
                suggestions.add(record["label"])

    driver.close()
    return sorted(list(suggestions))

def get_tpu_codes_for_names(names):
    uri = "neo4j+s://test-neo4j.manageai.co.th:7687"
    user = "neo4j"
    password = "iRU8ElhtDhoe0XdM72c0muzWY"

    driver = GraphDatabase.driver(uri, auth=(user, password))
    result_names = []
    result_codes = []

    with driver.session() as session:
        for name in names:
            res = session.run("""
                MATCH (d:DRUG)
                WHERE d.`TPUNAME` = $name
                RETURN d.`TPUNAME` AS name, d.`TMTID(TPU)` AS code
                LIMIT 1
            """, name=name)
            for record in res:
                result_names.append(record["name"])
                result_codes.append(record["code"])

    driver.close()
    return ", ".join(result_names), ", ".join(result_codes)

def fill_inputs_from_dropdown(selected, existing_names, existing_codes):
    if not selected:
        return existing_names, existing_codes  # ถ้าไม่มีอะไรเลือกใหม่ → คงของเดิมไว้

    # ดึงค่าใหม่ที่เลือก
    names, codes = get_tpu_codes_for_names(selected)

    # รวมกับของเดิม (ถ้ามี)
    combined_names = [x.strip() for x in (existing_names + ", " + names).split(",") if x.strip()]
    combined_codes = [x.strip() for x in (existing_codes + ", " + codes).split(",") if x.strip()]

    # ลบซ้ำ
    combined_names = list(dict.fromkeys(combined_names))
    combined_codes = list(dict.fromkeys(combined_codes))

    return ", ".join(combined_names), ", ".join(combined_codes)


In [ ]:
css = """
    #custom-border {
       border: 3px solid #45b6fe;
       border-radius: 10px;
       transition: 0.3s ease-in-out;
       padding: 5px;
    }
    #hr {
    border: none;
    border-top: 2px dashed #ccc;
    margin: 20px 0;
    }
"""
def format_drug_result(idx, validation, top_tpu_name, top_tpu_code, top_gpu_name, top_gpu_code,
                       top_vtm_name, top_vtm_code, subs_list, suggestions):
    color = "#d1e7dd" if validation == "APPROVED" else "#fff3cd"
    text = "#000000"
    validation_text = f"<span style='text-color:{text}; background-color:{color}; padding:2px 6px; border-radius:5px;'>{validation}</span>"

    text = f"""
### 🧪 ยาตัวที่ {idx}

**ผลการจับคู่:** {validation_text}

**TPU:** {top_tpu_name} (`{top_tpu_code}`)
**GPU:** {top_gpu_name} (`{top_gpu_code}`)
**VTM:** {top_vtm_name} (`{top_vtm_code}`)
**SUBS:** {', '.join(subs_list) if subs_list else 'N/A'}
"""

    if suggestions:
        text += "\n\n💡**คำแนะนำ: ยาอื่นที่อาจหมายถึง:**\n"
        for sug in suggestions:
            sug_name = sug.get("tpu_name", "N/A")
            sug_code = sug.get("tpu_code", "N/A")
            text += f"- {sug_name} (`{sug_code}`)\n"
    else:
        text += "<hr>"  # ✅ เพิ่มเฉพาะถ้าไม่มีคำแนะนำ

    return text

def sanitize_drug_name(name):
    if not isinstance(name, str):
        return ""
    name = name.strip()
    name = name.replace("/", "")  # ใช้ full-width slash ป้องกันการชนกับ logic parser
    name = name.replace("\\", "")  # full-width backslash
    return name


def search_drugs(drug_names_input, codes_input):
    try:
        # drug_names = [name.strip() for name in drug_names_input.split(",") if name.strip()]
        drug_names = [sanitize_drug_name(name) for name in drug_names_input.split(",") if name.strip()]
        drug_codes = [code.strip() for code in codes_input.split(",") if code.strip()]

        payload = {
            "drug_name": drug_names,
        }
        if drug_codes:
            payload["tpu_gpu_code"] = drug_codes

        response = requests.post(API_SEARCH_URL, json=payload)
        results = response.json()

        outputs = []
        top1_names = []
        top1_tpucodes = []

        # Group results by input drug_name
        grouped_results = {}
        for result in results:
            tpu_name = result.get("TPUNAME", "")
            matched = None
            for original in drug_names:
                if original.lower() in tpu_name.lower() or tpu_name.lower() in original.lower():
                    matched = original
                    break
            grouped_results.setdefault(matched or "UNKNOWN", []).append(result)

        idx = 1
        for drug_name in drug_names:
            group = grouped_results.get(drug_name, [])
            if not group:
                outputs.append(f"❌ ไม่พบข้อมูลสำหรับยา: {drug_name}")
                continue

            result = group[0]  # Top1
            validation = result.get("code_validation", "UNKNOWN")
            top_tpu_name = result.get("TPUNAME", "N/A")
            top_tpu_code = result.get("TMTID(TPU)", "N/A")
            top_gpu_name = result.get("GPUNAME", "N/A")
            top_gpu_code = result.get("TMTID(GPU)", "N/A")
            top_vtm_name = result.get("VTMNAME", "N/A")
            top_vtm_code = result.get("TMTID(VTM)", "N/A")
            subs_list = result.get("SUBS_LIST", [])
            suggestions = result.get("suggested_tpu", [])

            markdown_result = format_drug_result(
                idx, validation,
                top_tpu_name, top_tpu_code,
                top_gpu_name, top_gpu_code,
                top_vtm_name, top_vtm_code,
                subs_list, suggestions
            )

            outputs.append(markdown_result)
            top1_names.append(top_tpu_name)
            top1_tpucodes.append(top_tpu_code)
            idx += 1

        return outputs, ", ".join(top1_names), ", ".join(top1_tpucodes)

    except Exception as e:
        return [f"❌ Error: {str(e)}"], "", ""

In [ ]:
with gr.Blocks(css=css) as demo:
    with gr.Tabs():
        with gr.Tab("🔍 ค้นหายา"):
            with gr.Column():
                search_input = gr.Textbox(label="พิมพ์เพื่อค้นหายา", placeholder="เช่น aspirin, hydrochlorothiazide...")
                autocomplete_output = gr.Dropdown(label="รายการที่พบ", multiselect=True, interactive=True)

                # พิมพ์เพื่อค้นหาชื่อยา
                def fetch_suggestions(query):
                    return gr.update(choices=search_suggestions(query), value=[])

                search_input.change(fn=fetch_suggestions, inputs=[search_input], outputs=[autocomplete_output])

            with gr.Row():
                drug_names_input = gr.Textbox(label="ชื่อยา (คั่นด้วย , comma)", lines=2)
                codes_input = gr.Textbox(label="รหัสยา (คั่นด้วย , comma) [เว้นว่างได้]", lines=2)

            # with gr.Row():
            #     drug_names_input = gr.Textbox(label="ชื่อยา (คั่นด้วย , comma)", lines=2, value="BILDURETIC (บางกอกแล็ป แอนด์ คอสเมติค) (amiloride hydrochloride 5 mg + hydrochlorothiazide 50 mg) tablet, ASPIRIN COATED (โอสถ อินเตอร์ แลบบอราทอรีส์) (aspirin 300 mg) gastro-resistant, BEFARIN 3 (เบอร์ลินฟาร์มาซูติคอลอินดัสตรี้) (warfarin sodium 3 mg)")
            #     codes_input = gr.Textbox(label="รหัสยา (คั่นด้วย , comma) [เว้นว่างได้]", lines=2, value="262926, 226243, 108158")

            autocomplete_output.change(
                fn=fill_inputs_from_dropdown,
                inputs=[autocomplete_output, drug_names_input, codes_input],  # เพิ่ม inputs เดิม
                outputs=[drug_names_input, codes_input]
            )


            search_button = gr.Button("ค้นหา")

            with gr.Column():
                result_markdowns = [gr.Markdown(visible=False) for _ in range(5)]

                top1_drug_names = gr.Textbox(visible=False)
                top1_tpu_ids = gr.Textbox(visible=False)

            def update_ui(name_input, code_input):
                results, top1_names, top1_tpucodes = search_drugs(name_input, code_input)
                components = []
                for i in range(len(result_markdowns)):
                    if i < len(results):
                        components.append(gr.update(value=results[i], visible=True))
                    else:
                        components.append(gr.update(value="", visible=False))
                return components + [
                    gr.update(value=top1_names),  # อัปเดต drug_input
                    gr.update(value=top1_tpucodes)  # อัปเดต tpu_input
                ]

        # --- Tab 2: ตรวจสอบปฏิกิริยา ---
        with gr.Tab("💊 ตรวจสอบปฏิกิริยายา"):
            with gr.Row():
                # ใช้ตัวแปร shared drug_input, tpu_input จาก tab ก่อนหน้า
                drug_input = gr.Textbox(label="Input Drug Names (comma-separated)", lines=5)
                tpu_input = gr.Textbox(label="Input TPUIDs (comma-separated)", lines=5)

                search_button.click(
                    fn=update_ui,
                    inputs=[drug_names_input, codes_input],
                    outputs=result_markdowns + [drug_input, tpu_input]
                )
            submit_btn = gr.Button("Check Interactions")

            # Dropdown for displaying interactions
            gr.Markdown("### เลือกคู่ยาเพื่อดูรายละเอียดปฎิกิริยา")
            interaction_output = gr.Dropdown(
                label="Select Drug Interaction",
                choices=[],
                interactive=True,
                elem_id="custom-border"
            )

            with gr.Row():
                significance_box = gr.Textbox(label="Significance", interactive=False)
                onset_box = gr.Textbox(label="Onset", interactive=False)
                severity_box = gr.Textbox(label="Severity", interactive=False)
                documentation_box = gr.Textbox(label="Documentation", interactive=False)

            with gr.Row():
                with gr.Accordion("Effects", open=True):
                    effect_box = gr.Textbox(label="Effects", interactive=False, lines=1.5)

                with gr.Accordion("Mechanism", open=True):
                    mechanism_box = gr.Textbox(label="Mechanism", interactive=False, lines=1.5)

            with gr.Row():
                with gr.Accordion("Management", open=True):
                    management_box = gr.Textbox(label="Management", interactive=False, lines=1.5)

                with gr.Accordion("Discussion", open=True):
                    discussion_box = gr.Textbox(label="Discussion", interactive=False, lines=1.5)

            error_message_box = gr.Textbox(label="Error", interactive=False, visible=False)

            def prefill_and_submit(drug_names, tpuid_ids):
                return on_submit(drug_names, tpuid_ids)

            submit_btn.click(
                prefill_and_submit,
                inputs=[drug_input, tpu_input],  # ✅ ใช้ textbox จริง
                outputs=[
                    interaction_output,
                    error_message_box,
                    gr.State(),
                    significance_box,
                    onset_box,
                    severity_box,
                    documentation_box,
                    effect_box,
                    mechanism_box,
                    management_box,
                    discussion_box
                ]
            )

            interaction_output.select(
                on_interaction_select,
                inputs=[interaction_output, drug_input, tpu_input],  # ✅ ใช้ textbox จริง
                outputs=[
                    significance_box, onset_box, severity_box, documentation_box,
                    effect_box, mechanism_box, management_box, discussion_box
                ]
            )

demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://89da12b265d4302958.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [101]:
# 574098, 262926, 226243, 108159

# SARA (ไทยนครพัฒนา) (paracetamol 300 mg/2 mL) solution for injection, 2 mL vial,
# BILDURETIC (บางกอกแล็ป แอนด์ คอสเมติค) (amiloride hydrochloride 5 mg + hydrochlorothiazide 50 mg) tablet,
# ASPIRIN COATED (โอสถ อินเตอร์ แลบบอราทอรีส์) (aspirin 300 mg) gastro-resistant,
# BEFARIN 3 (เบอร์ลินฟาร์มาซูติคอลอินดัสตรี้) (warfarin sodium 3 mg)

In [ ]:
VERAPAMIL, RIFAMPICIdN, ATORVASTATIN

In [ ]:
456977 | 393047